# [VQ-VAE](https://arxiv.org/abs/1711.00937) by  [Aäron van den Oord](https://twitter.com/avdnoord) et al. in PyTorch

## Introduction

Variational Auto Encoders (VAEs) can be thought of as what all but the last layer of a neural network is doing, namely feature extraction or seperating out the data. Thus given some data we can think of using a neural network for representation generation. 

Recall that the goal of a generative model is to estimate the probability distribution of high dimensional data such as images, videos, audio or even text by learning the underlying structure in the data as well as the dependencies between the different elements of the data. This is very useful since we can then use this representation to generate new data with similar properties. This way we can also learn useful features from the data in an unsupervised fashion.

The VQ-VAE uses a discrete latent representation mostly because many important real-world objects are discrete. For example in images we might have categories like "Cat", "Car", etc. and it might not make sense to interpolate between these categories. Discrete representations are also easier to model since each category has a single value whereas if we had a continous latent space then we will need to normalize this density function and learn the dependencies between the different variables which could be very complex.

### Code

I have followed the code from the TensorFlow implementation by the author which you can find here [vqvae.py](https://github.com/deepmind/sonnet/blob/master/sonnet/python/modules/nets/vqvae.py) and [vqvae_example.ipynb](https://github.com/deepmind/sonnet/blob/master/sonnet/examples/vqvae_example.ipynb). 

Another PyTorch implementation is found at [pytorch-vqvae](https://github.com/ritheshkumar95/pytorch-vqvae).


## Basic Idea

The overall architecture is summarized in the diagram below:

![](images/vq-vae.png)

We start by defining a latent embedding space of dimension `[K, D]` where `K` are the number of embeddings and `D` is the dimensionality of each latent embeddng vector, i.e. $e_i \in \mathbb{R}^{D}$. The model is comprised of an encoder and a decoder. The encoder will map the input to a sequence of discrete latent variables, whereas the decoder will try to reconstruct the input from these latent sequences. 

More preciesly, the model will take in batches of RGB images,  say $x$, each of size 32x32 for our example, and pass it through a ConvNet encoder producing some output $E(x)$, where we make sure the channels are the same as the dimensionality of the latent embedding vectors. To calculate the discrete latent variable we find the nearest embedding vector and output it's index. 

The input to the decoder is the embedding vector corresponding to the index which is passed through the decoder to produce the reconstructed image. 

Since the nearest neighbour lookup has no real gradient in the backward pass we simply pass the gradients from the decoder to the encoder  unaltered. The intuition is that since the output representation of the encoder and the input to the decoder share the same `D` channel dimensional space, the gradients contain useful information for how the encoder has to change its output to lower the reconstruction loss.

## Loss

The total loss is actually composed of three components

1. **reconstruction loss**: which optimizes the decoder and encoder
1. **codebook loss**: due to the fact that gradients bypass the embedding, we use a dictionary learning algorithm  which uses an $l_2$  error to move the embedding vectors $e_i$ towards the encoder output
1. **commitment loss**:  since the volume of the embedding space is dimensionless, it can grow arbirtarily if the embeddings $e_i$ do not train as fast as  the encoder parameters, and thus we add a commitment loss to make sure that the encoder commits to an embedding

In [386]:
#!pip3 install -U -r requirements.txt

In [387]:
from __future__ import print_function


import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter


from six.moves import xrange

import umap

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

## Create run dir and init logging

In [388]:
from datetime import datetime
import os
import logging

run_id = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
run_dir = f'./runs/{run_id}'
os.makedirs(run_dir, exist_ok=True)

logger = logging.getLogger(f'{run_id}')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

fh = logging.FileHandler(filename=f'{run_dir}/info.log', encoding='utf-8')
fh.setLevel(logging.INFO)
logger.addHandler(fh)

fh = logging.FileHandler(filename=f'{run_dir}/debug.log', encoding='utf-8')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)


In [389]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Data

In [390]:
import gensim.downloader as api
model_name = 'glove-wiki-gigaword-50'
wv = api.load(model_name)

KeyboardInterrupt: 

In [ ]:
from torch.utils.data import Dataset, DataLoader

class WordEmbeddingDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk

all_data = wv.vectors[:20000]
pos = nltk.pos_tag(wv.index_to_key[:len(all_data)])
all_words = np.array(wv.index_to_key[:len(all_data)])
idx = np.array([i for i, v in enumerate(all_data) if pos[i][1] in ['NN','JJ','RB','VB'] and all_words[i] not in ['"', "n't", '_', '-', 'u.s.', '%', 'i']])
all_data = all_data[idx]
all_words = all_words[idx]
all_data_len = len(all_data)
training_data_len = int(all_data_len * 0.8)
training_data = WordEmbeddingDataset(all_data[:training_data_len])
validation_data = WordEmbeddingDataset(all_data[training_data_len:])

In [ ]:
data_variance = np.var((training_data.data - np.min(training_data.data))
                       / (np.max(training_data.data) - np.min(training_data.data)))

cosine_similarity_eps = 1e-50

## Vector Quantizer Layer

This layer takes a tensor to be quantized. The channel dimension will be used as the space in which to quantize. All other dimensions will be flattened and will be seen as different examples to quantize.

The output tensor will have the same shape as the input.

As an example for a `BCHW` tensor of shape `[16, 64, 32, 32]`, we will first convert it to an `BHWC` tensor of shape `[16, 32, 32, 64]` and then reshape it into `[16384, 64]` and all `16384` vectors of size `64`  will be quantized independently. In otherwords, the channels are used as the space in which to quantize. All other dimensions will be flattened and be seen as different examples to quantize, `16384` in this case.

In [ ]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        logger.debug(f'self._embedding.weight.data=\n{self._embedding.weight.data}\n')
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        inputs = inputs.contiguous()
        input_shape = inputs.shape

        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate cosine similarities
        repeated_input = flat_input.repeat(1, self._embedding.weight.size(0)).view(-1, self._embedding_dim)
        repeated_embedding = self._embedding.weight.repeat(flat_input.size(0), 1).view(-1, self._embedding_dim)
        cos = nn.CosineSimilarity(dim=1, eps=cosine_similarity_eps)
        sims = cos(repeated_input, repeated_embedding).view(-1, self._embedding.weight.size(0))

        # Encoding
        encoding_indices = torch.argmax(sims, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(flat_input.shape)
        
        # Loss
        e_latent_loss = F.cosine_embedding_loss(quantized.detach(), flat_input, torch.ones(quantized.size(0)))
        q_latent_loss = F.cosine_embedding_loss(quantized, flat_input.detach(), torch.ones(quantized.size(0)))
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = flat_input + (quantized - flat_input).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        return loss, quantized.contiguous(), perplexity, encodings

We will also implement a slightly modified version  which will use exponential moving averages to update the embedding vectors instead of an auxillary loss. This has the advantage that the embedding updates are independent of the choice of optimizer for the encoder, decoder and other parts of the architecture. For most experiments the EMA version trains faster than the non-EMA version.

In [ ]:
class VectorQuantizerEMA(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost, decay, epsilon=1e-5):
        super(VectorQuantizerEMA, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.normal_()
        logger.debug(f'self._embedding.weight.data=\n{self._embedding.weight.data}\n')
        self._commitment_cost = commitment_cost
        
        self.register_buffer('_ema_cluster_size', torch.zeros(num_embeddings))
        self._ema_w = nn.Parameter(torch.Tensor(num_embeddings, self._embedding_dim))
        self._ema_w.data.normal_()
        
        self._decay = decay
        self._epsilon = epsilon

    def forward(self, inputs):
        inputs = inputs.contiguous()

        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate cosine similarities
        repeated_input = flat_input.repeat(1, self._embedding.weight.size(0)).view(-1, self._embedding_dim)
        repeated_embedding = self._embedding.weight.repeat(flat_input.size(0), 1).view(-1, self._embedding_dim)
        cos = nn.CosineSimilarity(dim=1, eps=cosine_similarity_eps)
        sims = cos(repeated_input, repeated_embedding).view(-1, self._embedding.weight.size(0))

        # Encoding
        encoding_indices = torch.argmax(sims, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(flat_input.shape)
        
        # Use EMA to update the embedding vectors
        if self.training:
            self._ema_cluster_size = self._ema_cluster_size * self._decay + \
                                     (1 - self._decay) * torch.sum(encodings, 0)
            
            # Laplace smoothing of the cluster size
            n = torch.sum(self._ema_cluster_size.data)
            self._ema_cluster_size = (
                (self._ema_cluster_size + self._epsilon)
                / (n + self._num_embeddings * self._epsilon) * n)
            
            dw = torch.matmul(encodings.t(), flat_input)
            self._ema_w = nn.Parameter(self._ema_w * self._decay + (1 - self._decay) * dw)
            
            self._embedding.weight = nn.Parameter(self._ema_w / self._ema_cluster_size.unsqueeze(1))
        
        # Loss
        e_latent_loss = F.cosine_embedding_loss(quantized.detach(), flat_input, torch.ones(quantized.size(0)))
        loss = self._commitment_cost * e_latent_loss
        
        # Straight Through Estimator
        quantized = flat_input + (quantized - flat_input).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        return loss, quantized.contiguous(), perplexity, encodings

## Encoder & Decoder Architecture

The encoder and decoder architecture is based on a ResNet and is implemented below:

In [ ]:
class Residual(nn.Module):
    def __init__(self, num_hiddens, num_residual_hiddens):
        super(Residual, self).__init__()
        self._block = nn.Sequential(
            nn.ReLU(True),
            nn.Linear(in_features=num_hiddens,
                      out_features=num_residual_hiddens),
            nn.ReLU(True),
            nn.Linear(in_features=num_residual_hiddens,
                      out_features=num_hiddens)
        )
    
    def forward(self, x):
        return x + self._block(x)


class ResidualStack(nn.Module):
    def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens):
        super(ResidualStack, self).__init__()
        self._num_residual_layers = num_residual_layers
        self._layers = nn.ModuleList([Residual(num_hiddens, num_residual_hiddens)
                             for _ in range(self._num_residual_layers)])

    def forward(self, x):
        for i in range(self._num_residual_layers):
            x = self._layers[i](x)
        return F.relu(x)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens, embedding_dim):
        super(Encoder, self).__init__()

        self._lin_1 = nn.Linear(in_features=embedding_dim,
                                out_features=num_hiddens)

        self._lin_2 = nn.Linear(in_features=num_hiddens,
                                out_features=num_hiddens)

        self._lin_3 = nn.Linear(in_features=num_hiddens,
                                out_features=num_hiddens)

        self._residual_stack = ResidualStack(num_hiddens=num_hiddens,
                                             num_residual_layers=num_residual_layers,
                                             num_residual_hiddens=num_residual_hiddens)

    def forward(self, inputs):
        x = self._lin_1(inputs)
        x = F.relu(x)
        
        x = self._lin_2(x)
        x = F.relu(x)
        
        x = self._lin_3(x)
        return self._residual_stack(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, embedding_dim, definition_len):
        super(Decoder, self).__init__()
        self._embedding_dim = embedding_dim
        self._definition_len = definition_len

    def forward(self, inputs):
        definition = inputs.view((-1, self._definition_len, self._embedding_dim))
        recon = torch.mean(definition, 1)
        return recon, definition

## Train

We use the hyperparameters from the author's code:

In [ ]:
batch_size = 256
num_training_updates = 15000

num_hiddens = 128
num_residual_hiddens = 32
num_residual_layers = 2

embedding_dim = 50
num_embeddings = 100
definition_len = 3

commitment_cost = 0.25

decay = 0.99

learning_rate = 1e-3

logger.info(f'model_name={model_name}')
logger.info(f'batch_size={batch_size}')
logger.info(f'num_training_updates={num_training_updates}')
logger.info(f'num_hiddens={num_hiddens}')
logger.info(f'num_residual_hiddens={num_residual_hiddens}')
logger.info(f'embedding_dim={embedding_dim}')
logger.info(f'num_embeddings={num_embeddings}')
logger.info(f'definition_len={definition_len}')
logger.info(f'commitment_cost={commitment_cost}')
logger.info(f'decay={decay}')
logger.info(f'learning_rate={learning_rate}')
logger.info(f'cosine_similarity_eps={cosine_similarity_eps}')
logger.info('')

In [ ]:
training_loader = DataLoader(training_data, 
                             batch_size=batch_size, 
                             shuffle=True,
                             pin_memory=True)

In [ ]:
validation_loader = DataLoader(validation_data,
                               batch_size=32,
                               shuffle=True,
                               pin_memory=True)

In [ ]:
class Model(nn.Module):
    def __init__(self, num_hiddens, definition_len, num_residual_layers, num_residual_hiddens,
                 num_embeddings, embedding_dim, commitment_cost, decay=0):
        super(Model, self).__init__()
        
        self._encoder = Encoder(num_hiddens,
                                num_residual_layers, 
                                num_residual_hiddens,
                                embedding_dim)
        self._pre_vq_lin = nn.Linear(in_features=num_hiddens,
                                      out_features=embedding_dim * definition_len)
        if decay > 0.0:
            self._vq_vae = VectorQuantizerEMA(num_embeddings, embedding_dim,
                                              commitment_cost, decay)
        else:
            self._vq_vae = VectorQuantizer(num_embeddings, embedding_dim,
                                           commitment_cost)
        self._decoder = Decoder(embedding_dim, definition_len)

    def forward(self, x):
        z = self._encoder(x)
        z = self._pre_vq_lin(z)
        loss, quantized, perplexity, _ = self._vq_vae(z)
        x_recon, x_definition = self._decoder(quantized)

        return loss, x_recon, x_definition, perplexity

In [ ]:
model = Model(num_hiddens, definition_len, num_residual_layers, num_residual_hiddens,
              num_embeddings, embedding_dim, 
              commitment_cost, decay).to(device)

logger.info(f'model=\n{model}')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, amsgrad=False)

In [ ]:
model.train()
train_res_recon_error = []
train_vq_loss = []
train_res_perplexity = []

for i in xrange(num_training_updates):
    data = next(iter(training_loader))
    data = data.to(device)
    optimizer.zero_grad()

    vq_loss, data_recon, data_definition, perplexity = model(data)
    recon_error = F.cosine_embedding_loss(data_recon, data, torch.ones(data_recon.size(0))) / data_variance
    loss = recon_error + vq_loss
    loss.backward()

    optimizer.step()
    
    train_res_recon_error.append(recon_error.item())
    train_vq_loss.append(vq_loss.item())
    train_res_perplexity.append(perplexity.item())

    if (i+1) % 100 == 0:
        logger.debug('%d iterations' % (i+1))
        logger.debug('recon_error: %.3f' % np.mean(train_res_recon_error[-100:]))
        logger.debug('vq_loss: %.3f' % np.mean(train_vq_loss[-100:]))
        logger.debug('perplexity: %.3f' % np.mean(train_res_perplexity[-100:]))
        logger.debug('')

# Write final result
logger.info('%d iterations' % (i+1))
logger.info('recon_error: %.3f' % np.mean(train_res_recon_error[-100:]))
logger.info('vq_loss: %.3f' % np.mean(train_vq_loss[-100:]))
logger.info('perplexity: %.3f' % np.mean(train_res_perplexity[-100:]))
logger.info('')

## Plot Loss

In [ ]:
train_res_recon_error_smooth = savgol_filter(train_res_recon_error, 201, 7)
train_res_vq_loss_smooth = savgol_filter(train_vq_loss, 201, 7)
train_res_perplexity_smooth = savgol_filter(train_res_perplexity, 201, 7)

In [ ]:
f = plt.figure(figsize=(16,8))
ax = f.add_subplot(2,2,1)
ax.plot(train_res_recon_error_smooth)
ax.set_yscale('log')
ax.set_title('Smoothed cosine embedding loss.')
ax.set_xlabel('iteration')

ax = f.add_subplot(2,2,2)
ax.plot(train_res_vq_loss_smooth)
ax.set_yscale('log')
ax.set_title('Smoothed VQ loss.')
ax.set_xlabel('iteration')

ax = f.add_subplot(2,2,3)
ax.plot(train_res_perplexity_smooth)
ax.set_title('Smoothed Average codebook usage (perplexity).')
ax.set_xlabel('iteration')

plt.savefig(f'{run_dir}/loss.png')

## Auxilary functions

In [ ]:
cosine_similarity = nn.CosineSimilarity(dim=0, eps=cosine_similarity_eps)

def closest_vector(v):
    i = np.argmax([cosine_similarity(v, torch.tensor(w).detach()) for w in all_data])
    return all_data[i]

def closest_word(v):
    i = np.argmax([cosine_similarity(v, torch.tensor(w).detach()) for w in all_data])
    return all_words[i]

def tensor_to_closest_word(t):
    return closest_word(t)

def tensor_to_embedding_closest_word(v):
    i = np.argmax([cosine_similarity(v, w) for w in model._vq_vae._embedding.weight.detach()])
    return tensor_to_closest_word(model._vq_vae._embedding.weight.detach()[i])

## View Reconstructions

In [ ]:
model.eval()

valid_originals = next(iter(validation_loader))
valid_originals = valid_originals.to(device)

vq_output_eval = model._pre_vq_lin(model._encoder(valid_originals))
_, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
valid_reconstructions, valid_definition = model._decoder(valid_quantize)

for i in range(10):
    orig_word = tensor_to_closest_word(valid_originals[i].detach())
    recon_word = tensor_to_closest_word(valid_reconstructions[i].detach())
    def_text = ' '.join([tensor_to_embedding_closest_word(v.detach()) for v in valid_definition[i]])
    logger.info(f'{orig_word} to {recon_word}: {def_text}')

In [ ]:
train_originals = next(iter(training_loader))
train_originals = train_originals.to(device)

vq_output_eval = model._pre_vq_lin(model._encoder(train_originals))
_, train_quantize, _, _ = model._vq_vae(vq_output_eval)
train_reconstructions, train_definition = model._decoder(train_quantize)

for i in range(10):
    orig_word = tensor_to_closest_word(train_originals[i].detach())
    recon_word = tensor_to_closest_word(train_reconstructions[i].detach())
    def_text = ' '.join([tensor_to_embedding_closest_word(v.detach()) for v in train_definition[i]])
    logger.info(f'{orig_word} to {recon_word}: {def_text}')

## View Embedding

In [ ]:
proj = umap.UMAP(n_neighbors=3,
                 min_dist=0.1,
                 metric='cosine').fit_transform(model._vq_vae._embedding.weight.data.cpu())

In [ ]:
plt.scatter(proj[:,0], proj[:,1], alpha=0.3)
plt.savefig(f'{run_dir}/embedding.png')

In [ ]:
for i in range(num_embeddings):
    logger.info(f'{i}:\t{tensor_to_closest_word(model._vq_vae._embedding.weight.data[i])} {model._vq_vae._embedding.weight.data[i].tolist()}')

In [ ]:
logging.shutdown()